In [ ]:
import dedalus_sphere.jacobi as Jacobi

from IPython.display import display, Markdown
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=4,suppress=True)


In [ ]:
m, ell, alpha, sigma = 2, 5, 1, 0
Nmax = 20

t, w = Jacobi.quadrature(Nmax, ell+alpha+1/2, m+sigma)
polys = Jacobi.polynomials(Nmax, ell+alpha+1/2, m+sigma, t)
nrm = np.sqrt(np.sum(w*polys**2, axis=1))
print(nrm)


In [ ]:
m, ell, alpha, sigma = 1, 2, 3, 0
Nmax = 20

# t  = 2*s**2 - 1
# dt = 4*s * ds
# dV = s*(1-s**2)**(1/2) * ds
# dμ = (1-s**2)**alpha * dV
#    = s*(1-s**2)**(alpha + 1/2) * ds
#    = 1/4 * (1/2)**(alpha + 1/2) * (1-t)**(alpha + 1/2) * dt
#    = (1/2)**(2 + alpha + 1/2) * (1-t)**(alpha + 1/2) * dt

# Nodes and weights for dμ' = (1-t)**(alpha+1/2) dt, which is incorrectly scaled,
# then scale the weights for dμ
t, w = Jacobi.quadrature(Nmax, alpha+1/2, 0)
dμ = (1/2)**(2+alpha+1/2) * w

polys = Jacobi.polynomials(Nmax-(ell+m)//2, ell+alpha+1/2, m+sigma, t)

# Psi(s) and Psi(t) are equivalent but expressed in different variables
# Psi is not normalized with respect to our measure dμ
s = np.sqrt((1+t)/2)
psis = 2**((2+ell+alpha+1/2+m+sigma)/2) * s**(m+sigma) * (1-s**2)**(ell/2) * polys
psit = 2**((2+alpha+1/2)/2) * (1+t)**((m+sigma)/2) * (1-t)**(ell/2) * polys
psi = (1+t)**((m+sigma)/2) * (1-t)**(ell/2) * polys

assert np.max(abs(psis-psit)) < 1e-12

nrms = np.sqrt(np.sum(dμ*psis**2, axis=1))  # Orthonormal polys w.r.t. dμ = dμ(s)
nrmt = np.sqrt(np.sum(dμ*psit**2, axis=1))  # Orthonormal polys w.r.t. dμ = dμ(t)
nrm = np.sqrt(np.sum(w*psi**2, axis=1))     # Orthonormal polys w.r.t. (1-t)**(alpha+1/2)*dt (rescaled measure)

assert np.max(abs(nrms-1)) < 1e-15
assert np.max(abs(nrmt-1)) < 1e-15
assert np.max(abs(nrm-1)) < 1e-15

# All the projections
N = Nmax-(ell+m)//2
proj_ij = np.asarray([[np.sum(w*psi[i,:]*psi[j,:]) for i in range(N)] for j in range(N)])
assert np.max(abs(proj_ij - np.eye(N))) < 1e-14

scale = lambda a: 2**((2+a+1/2)/2)
alphas = np.linspace(-1,4.2,100)
fig, ax = plt.subplots()
ax.plot(alphas, scale(alphas))
for a in range(5):
    s = scale(a)
    ax.plot([-1,a], [s,s],'--k')
    ax.plot([a,a], [scale(alphas[0]),s],'--k')
    ax.text(-1,s+.25,f'{a}:  {s:1.4f}')
ax.set_xlabel('alpha')
ax.set_ylabel('t-normalization factor');


def norm_ratio(da):
    """Ratio of norms for converting from Psi(alpha) -> Psi(alpha+da)"""
    return 2**(da/2)


In [ ]:
# Projections for sphere basis in s
ell, alpha = 2, 3
Nmax = 10
N = Nmax-ell//2

t, w = Jacobi.quadrature(Nmax, alpha, 1/2)
w *= 4
s = np.sqrt((1+t)/2)
Q = s**ell * Jacobi.polynomials(N, alpha, ell+1/2, t)

c = np.zeros(N)
c[0] = 2
c[1] = 4
c[3] = -1
f = (Q.T @ c)

proj_ij = np.asarray([[np.sum(w*Q[i,:]*Q[j,:]) for i in range(N)] for j in range(N)])
assert np.max(abs(proj_ij - np.eye(N))) < 1e-14

Fn = np.asarray([np.sum(w*Q[i,:]*f) for i in range(N)])
assert np.max(abs(Fn - c)) < 1e-14


In [ ]:
# 2D Projections for sphere basis
m, Lmax, Nmax, alpha, sigma = 2, 5, 8, 3/2, -1

def fun(t, eta):
    ell, k = 3, 5
    coeffs = np.random.random((Lmax,Nmax))
    Peta = Jacobi.polynomials(Lmax, alpha, alpha, eta)
    Pt = Jacobi.polynomials(Nmax, ell+alpha+1/2, m+sigma, t)
    return (1+t)**((m+sigma)/2) * (1-t)**(ell/2) * Peta[ell][:,np.newaxis] * Pt[k][np.newaxis,:]

true_coeffs = np.random.random((Lmax,Nmax))
def random_fun(t, eta):
    Peta = Jacobi.polynomials(Lmax, alpha, alpha, eta)
    f = 0*t[np.newaxis,:] * eta[:,np.newaxis]
    for ell in range(Lmax):
        Pt = (1+t)**((m+sigma)/2) * (1-t)**(ell/2) * Jacobi.polynomials(Nmax, ell+alpha+1/2, m+sigma, t)
        for k in range(Nmax):
            f += true_coeffs[ell,k] * Peta[ell][:,np.newaxis] * Pt[k][np.newaxis,:]
    return f    

def project(f, m, Lmax, Nmax, alpha, sigma):
    ratio = 1
    coeffs = np.zeros((Lmax, Nmax))
    eta, weta = Jacobi.quadrature(ratio*Lmax, alpha, alpha)
    Peta = Jacobi.polynomials(Lmax, alpha, alpha, eta)
    weta = weta[:,np.newaxis]
    for ell in range(Lmax):
        t, wt = Jacobi.quadrature(Nmax, ell+alpha+1/2, m+sigma)
        Pt = (1+t)**(-(m+sigma)/2) * (1-t)**(-ell/2) * Jacobi.polynomials(Nmax, ell+alpha+1/2, m+sigma, t)
#         t, wt = Jacobi.quadrature(Nmax+(m+sigma)//2+2, alpha+1/2, 0)
#         Pt = (1+t)**((m+sigma)/2) * (1-t)**(ell/2) * Jacobi.polynomials(Nmax, ell+alpha+1/2, m+sigma, t)
        coeffs[ell,:] = [np.sum(weta * wt[np.newaxis,:] * Peta[ell][:,np.newaxis] * Pt[k] * f(t,eta)) for k in range(Nmax)]
    return coeffs
    
coeffs = project(random_fun, m, Lmax, Nmax, alpha, sigma)

np.set_printoptions(precision=16,suppress=False)
print(coeffs-true_coeffs)

np.set_printoptions(precision=4,suppress=True)
print(np.max(abs(coeffs-true_coeffs)))


In [ ]:
f = lambda x: np.sqrt(x+1)
# f = lambda x: (x+1)**2

Lmax = 30
t, w = Jacobi.quadrature(Lmax, 0, 1/2)
P = (1+t)**(-1/2) * Jacobi.polynomials(Lmax, 0, 1/2, t)

feval = f(t)
coeffs = np.zeros(Lmax)
for ell in range(Lmax):
    coeffs[ell] = np.sum(w * P[ell] * feval)

plt.figure()
plt.semilogy(abs(coeffs))

In [ ]:
# Let's modify the sphere basis with a non-vanishing function of t.
# We have Q(t) = (1-t)**(a*ell) * (1+t)**(b*ell + m/2) * h(t)**ell * P_{n}^{((2*ell+1)*a, (2*ell+1)*b+m)}(t)
# We take the quadrature nodes to be those for weight function w(t) = (1-t)**a * (1+t)**b.
# The basis is NOT orthonormal - only Q_{n}/h forms an orthonormal set.
# h(t) in general induces the weight function of a semi-classical orthogonal polynomial.
# We project onto our Q basis by forming < f/(h**ell), Q/(h**ell) > where we necessarily divide
# by the non-Jacobi height function in both inputs to the integral.
m, ell, alpha = 4, 2, 3
a, b = 1/2, 0
Nmax = 20
N = Nmax-(m+ell)//2

t, w = Jacobi.quadrature(Nmax, a, b)
h = 2+t  # Some non-vanishing height function
Q = (1+t)**(b*ell+m/2)*(1-t)**(a*ell)*h**ell * Jacobi.polynomials(N, (2*ell+1)*a, (2*ell+1)*b+m, t)

c = np.zeros(N)
c[0] = 2
c[1] = 4
c[3] = -1
c[7] = 0.04
f = (Q.T @ c)

proj_ij = np.asarray([[np.sum(w*Q[i,:]*Q[j,:]/h**(2*ell)) for i in range(N)] for j in range(N)])
assert np.max(abs(proj_ij - np.eye(N))) < 1e-14

Fn = np.asarray([np.sum(w*Q[i,:]*f/h**(2*ell)) for i in range(N)])
assert np.max(abs(Fn - c)) < 1e-14


In [ ]:
m, Lmax, Nmax = 3, 5, 9

B = Jacobi.operator('B')
H = (1 + B(-1)@B(+1))

t, phi, eta = np.linspace(-1,1,100), np.array([0.]), np.linspace(-1,1,31)
P = Jacobi.polynomials(Lmax, 0, 0, eta).T
Q = Jacobi.polynomials(Nmax, 0, m, t)
h = (2+t)
t, h = t[np.newaxis,:], h[np.newaxis,:]
fmlk = lambda m,ell,k: (1+t)**(m/2) * P[:,ell][:,np.newaxis] * (h**ell * Q[k,:][np.newaxis,:])
expand = lambda F: np.sum([F[ell,k] * fmlk(m,ell,k) for ell in range(Lmax) for k in range(Nmax)],axis=0)

if True:
    F = np.random.random((Lmax,Nmax))
else:
    F = np.zeros((Lmax,Nmax))
    F[0,0] = 1
    F[0,3] = 2
    F[1,1] = 3
    F[1,2] = 4
    F[2,2] = 5
f = expand(F)

# Construct the boundary evaluation operator
Nmaxout = (H**(Lmax-1)).codomain(Nmax,0,m)[0]
Qout = Jacobi.polynomials(Nmaxout, 0, m, t.ravel())

Boundary = np.zeros((Nmaxout,Lmax*Nmax))
for ell in range(Lmax):
    nout = (H**ell).codomain(Nmax,0,m)[0]
    Boundary[:nout,ell*Nmax:(ell+1)*Nmax] = (P[-1,ell] * (H**ell)(Nmax,0,m)).todense()
boundary = Qout.T @ Boundary @ F.ravel()
boundary *= (1+t.ravel())**(m/2)

error = f[-1,:]-boundary
assert np.max(abs(error)) < 1e-12

fig, ax = plt.subplots(1,2,figsize=(9,4))
ax[0].plot(t.ravel(),f[-1,:],label='grid')
ax[0].plot(t.ravel(),boundary,label='op')
ax[1].plot(t.ravel(),error,label='op-grid')
ax[0].set_title('boundary')
ax[1].set_title('error')
for a in ax: 
    a.grid(True)
    a.legend()

# Plot a random vector in the null space
c = np.random.random(nullity)
F = (null_space @ c).reshape(Lmax,Nmax)
f = expand(F)
fig, ax = plt.subplots()
ax.plot(t.ravel(),f[-1,:])
ax.set_title('random vector in  null space')
ax.grid(True)
